# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.30.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProject'
project_folder = './capstone-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
CapstoneProject,quick-starts-ws-148168,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
# Setting up the compute cluster

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "CapstoneAML"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           min_nodes=1, # for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded...............
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
found = False
key = "HeartFailureRate"
description_text = "Dataset for heart failure recommendation"

from azureml.core import Dataset

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['../heart_failure_clinical_records_dataset.csv'],
                       target_path = 'train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key]
        df = dataset.to_pandas_dataframe()

else:
        dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/heart_failure_clinical_records_dataset.csv')])
        # preview the first 3 rows of the dataset
        df= dataset.to_pandas_dataframe()


df.describe()



Uploading an estimated of 1 files
Uploading ../heart_failure_clinical_records_dataset.csv
Uploaded ../heart_failure_clinical_records_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

I have chosen experimentation time out as the dataset is smaller and it can be covered within 20 minutes. 
The max concurrent iteration is 5 ( 1 less than total number of compute nodes in the cluster). 
The primary_metric is chosen as AUC_weighted orginally to handle balanced data better. 


In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output=True)

Submitting remote run.
No run_configuration provided, running on CapstoneAML with default configuration
Running on remote compute: CapstoneAML


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProject,AutoML_1997416e-92ca-40de-85ee-6cb021542604,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [8]:
# waiting for the run 

remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProject,AutoML_1997416e-92ca-40de-85ee-6cb021542604,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_1997416e-92ca-40de-85ee-6cb021542604',
 'target': 'CapstoneAML',
 'status': 'Completed',
 'startTimeUtc': '2021-06-29T13:49:36.095641Z',
 'endTimeUtc': '2021-06-29T14:03:08.144839Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'CapstoneAML',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneProject","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-148168","workspace_name":"quick-starts-ws-148168","region":"southcentralus","compute_target":"CapstoneAML","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"accuracy"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# Best model

# Retrieve best model from remote Run
#best_model_output = remote_run.get_pipeline_output(best_model_output_name)
best_run, model = remote_run.get_output()


Package:azureml-automl-runtime, training version:1.31.0, current version:1.30.0
Package:azureml-core, training version:1.31.0, current version:1.30.0
Package:azureml-dataprep, training version:2.18.0, current version:2.15.1
Package:azureml-dataprep-native, training version:36.0.0, current version:33.0.0
Package:azureml-dataprep-rslex, training version:1.16.0, current version:1.13.0
Package:azureml-dataset-runtime, training version:1.31.0, current version:1.30.0
Package:azureml-defaults, training version:1.31.0, current version:1.30.0
Package:azureml-interpret, training version:1.31.0, current version:1.30.0
Package:azureml-mlflow, training version:1.31.0, current version:1.30.0
Package:azureml-pipeline-core, training version:1.31.0, current version:1.30.0
Package:azureml-telemetry, training version:1.31.0, current version:1.30.0
Package:azureml-train-automl-client, training version:1.31.0, current version:1.30.0
Package:azureml-train-automl-runtime, training version:1.31.0, current ver

In [10]:
model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0.4166666666666667, reg_lambda=0.20833333333333334, subsample=0.5, tree_method='auto'))], verbose=False)), ('13', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('extratreesclassifier', ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, criterion='gini', max_depth=None, max_features=0.7, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.035789473684210524, min_samples_split=0.01, min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1, oob_score=True, random_state=None

In [11]:
model.steps

[('datatransformer',
  DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(
      estimators=[('18', Pipeline(memory=None,
               steps=[('robustscaler',
                       RobustScaler(copy=True, quantile_range=[25, 75],
                                    with_centering=True, with_scaling=False)),
                      ('extratreesclassifier',
                       ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                            class_weight=None, criterion='entropy',
                                            max_depth=None, max_features=0.8,
                                            max_leaf_nodes=None, max_samples=No

In [13]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#data-featurization

model_name = best_run.properties['model_name']
description = 'AutoML forecast example'
tags = None

model = remote_run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)

In [14]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-148168', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-148168'), name=AutoML1997416e938, id=AutoML1997416e938:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
# create inference_config
# trying default environment instead of env.yml file
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script="score.py")

TODO: In the cell below, send a request to the web service you deployed to test it.

In [18]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)


#deploy the model
service=model.deploy(workspace=ws,
                    name="heart-failure",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-29 14:15:35+00:00 Creating Container Registry if not exists..
2021-06-29 14:15:47+00:00 Generating deployment configuration.
2021-06-29 14:15:48+00:00 Submitting deployment to compute.
2021-06-29 14:15:51+00:00 Checking the status of deployment heart-failure..
2021-06-29 14:21:02+00:00 Checking the status of inference endpoint heart-failure.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [19]:
print(service.get_logs())

2021-06-29T14:20:54,875287500+00:00 - iot-server/run 
2021-06-29T14:20:54,876057800+00:00 - rsyslog/run 
2021-06-29T14:20:54,885521500+00:00 - gunicorn/run 
2021-06-29T14:20:54,955684700+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_429e58b1641c78c2352efc8ad21c49d9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_429e58b1641c78c2352efc8ad21c49d9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_429e58b1641c78c2352efc8ad21c49d9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_429e58b1641c78c2352efc8ad21c49d9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_429e58b1641c78c2352efc8ad21c49d9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

Test the Model

In [20]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')

print(service.scoring_uri)



service state: Healthy

scoring URI: 
http://7799d541-3f89-414c-b8fc-9819557262b2.southcentralus.azurecontainer.io/score

swagger URI: 
http://7799d541-3f89-414c-b8fc-9819557262b2.southcentralus.azurecontainer.io/swagger.json

http://7799d541-3f89-414c-b8fc-9819557262b2.southcentralus.azurecontainer.io/score
http://7799d541-3f89-414c-b8fc-9819557262b2.southcentralus.azurecontainer.io/swagger.json


In [21]:
import requests
import json

data={"data":
  [{'age': 87.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 981,
  'diabetes': 0,
  'ejection_fraction': 55,
  'high_blood_pressure': 1,
  'platelets': 368000,
  'serum_creatinine': 3.5,
  'serum_sodium': 137,
  'sex': 0,
  'smoking': 0,
  'time': 4}]
  }

input_data=json.dumps(data)

with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("The output is: ",result.json())

result.status_code

The output is:  Must pass 2-d input


200